# Transmon with tunable couplers

This notebook is similar but not as detail as `examples/fluxonium_multipath_coupling/fluxonium_mpcoupling_6q_x.ipynb`. Please check that notebook for more info.

This example is based on Xu, Y. et al. High-Fidelity, High-Scalability Two-Qubit Gate Scheme for Superconducting Qubits. Phys. Rev. Lett. 125, 240503 (2020). We construct a 5 transmon qubit system, where 3 of them (`q0`, `q2`, `q4`) are computational qubits and the other 2 (`q1`, `q3`) are the couplers. More concretely, the couplers are always initialized to $|0\rangle$ before the time evolution, and it is a leakage error if it is not in $|0\rangle$ afterwards. The computed unitary is only for the 3 qubits (`q0`, `q2`, `q4`).
The target unitary is simultaneous X gate $X^{\otimes 3}$

In [1]:
import jax.numpy as jnp
import haiku as hk

import supergrad
from supergrad.helper import Evolve
from supergrad.utils import compute_fidelity_with_1q_rotation_axis
from supergrad.utils.gates import sigmax
from supergrad.utils.optimize import scipy_minimize, adam_opt

from supergrad.scgraph.graph_transmon_1d import XGateTmon1D

truncated_dim = 5
add_random = True
share_params = False
compensation_option = 'only_vz'
opt = 'adam'

gate_graph = XGateTmon1D(seed=1)
qubit_subset = gate_graph.subgraph(['q0', 'q1', 'q2', 'q3', 'q4'])
evo = Evolve(qubit_subset,
             truncated_dim,
             add_random,
             share_params,
             idle_subsystem=['q1', 'q3'],  # these two qubits are treated as coupler
             compensation_option=compensation_option,
             options={
                 'astep': 5000,
                 'trotter_order': 2
             })
unitary = supergrad.tensor(*([sigmax()] * 3))  # the target unitary X^3


For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [2]:
def infidelity(params, unitary):
    params = hk.data_structures.merge(evo.all_params, params)

    sim_u = evo.eigen_basis(params)
    fidelity_vz, _ = compute_fidelity_with_1q_rotation_axis(unitary,
                                                            sim_u,
                                                            opt_method=None)

    return jnp.log10(1 - fidelity_vz)


params = {
    'q0_pulse_cos': {
        'amp': jnp.array(0.0922632),
        'length': jnp.array(39.99841052),
        'omega_d': jnp.array(31.89213402),
        'phase': jnp.array(-0.06459036)
    },
    'q2_pulse_cos': {
        'amp': jnp.array(0.10390872),
        'length': jnp.array(39.92211365),
        'omega_d': jnp.array(27.99554391),
        'phase': jnp.array(0.05805683)
    },
    'q4_pulse_cos': {
        'amp': jnp.array(0.09196213),
        'length': jnp.array(39.88357291),
        'omega_d': jnp.array(31.97277349),
        'phase': jnp.array(-0.07858071)
    },
}


Show the optimization procedure of control parameters

In [3]:
adam_opt(infidelity, params, (unitary,), {'adam_lr': 1e-6, 'steps': 3})


{'adam_lr': 1e-06, 'adam_lr_decay_rate': 1000, 'steps': 3, 'adam_b1': 0.9, 'adam_b2': 0.999}
step: 0
parameters:
{ 'q0_pulse_cos': { 'amp': 0.0922632,
                    'length': 39.99841052,
                    'omega_d': 31.89213402,
                    'phase': -0.06459036},
  'q2_pulse_cos': { 'amp': 0.10390872,
                    'length': 39.92211365,
                    'omega_d': 27.99554391,
                    'phase': 0.05805683},
  'q4_pulse_cos': { 'amp': 0.09196213,
                    'length': 39.88357291,
                    'omega_d': 31.97277349,
                    'phase': -0.07858071}}
gradient:
{ 'q0_pulse_cos': { 'amp': -0.00026231208687997333,
                    'length': 0.001054256036531395,
                    'omega_d': 0.000506687695474707,
                    'phase': -0.00030852529727553387},
  'q2_pulse_cos': { 'amp': 7.925136207858003e-05,
                    'length': -0.00037554374811209207,
                    'omega_d': 0.001345949844044239,
  

{'q0_pulse_cos': {'amp': Array(0.09226275, dtype=float64),
  'length': Array(39.99841096, dtype=float64),
  'omega_d': Array(31.89213446, dtype=float64),
  'phase': Array(-0.06458785, dtype=float64)},
 'q2_pulse_cos': {'amp': Array(0.10390917, dtype=float64),
  'length': Array(39.92211664, dtype=float64),
  'omega_d': Array(27.99554433, dtype=float64),
  'phase': Array(0.0580594, dtype=float64)},
 'q4_pulse_cos': {'amp': Array(0.0919617, dtype=float64),
  'length': Array(39.88356991, dtype=float64),
  'omega_d': Array(31.97277394, dtype=float64),
  'phase': Array(-0.07858026, dtype=float64)}}